In [ ]:
# Install the 3rd party packages
!pip install pylas  numpy tqdm

In [ ]:
# Install the 3rd party packages - for reading *.LAZ files (sometime gives error using laspy)
!pip install laspy lazrs pdal

In [ ]:
# Mount and connect to the Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the main function
import laspy
import numpy as np
import os
from tqdm import tqdm

# Function to perform re-tiling of a LAS file
def tile_lidar_las(las_file, output_dir, tile_size=100):
    """
    Splits a large LAS file into smaller 100m x 100m tiles.

    Parameters:
        las_file (str): Path to the large input LAS file.
        output_dir (str): Directory to save tiled LAS files.
        tile_size (int): The size of each tile (default: 100m x 100m).
    """
    # Load the LAS file
    print(f"Reading {las_file}...")
    las = laspy.read(las_file)

    # Extract point cloud data
    x, y, z = las.x, las.y, las.z
    min_x, max_x = x.min(), x.max()
    min_y, max_y = y.min(), y.max()

    # Create output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)

    # Determine number of tiles
    x_steps = np.arange(min_x, max_x, tile_size)
    y_steps = np.arange(min_y, max_y, tile_size)

    print(f"Generating {len(x_steps) * len(y_steps)} tiles...")

    # Iterate through tiles and extract points
    for i, x_min in tqdm(enumerate(x_steps), total=len(x_steps)):
        for j, y_min in enumerate(y_steps):
            x_max = x_min + tile_size
            y_max = y_min + tile_size

            # Filter points within the tile boundaries
            mask = (x >= x_min) & (x < x_max) & (y >= y_min) & (y < y_max)
            if np.sum(mask) == 0:
                continue  # Skip empty tiles

            # Create a new LAS object with selected points
            header = las.header  # Copy original header
            tile = laspy.create(point_format=las.point_format, file_version=las.header.version)
            tile.header = header
            tile.points = las.points[mask]

            # Save the tile
            tile_filename = os.path.join(output_dir, f"tile_{i}_{j}.las")
            tile.write(tile_filename)

    print(f"Tiles saved in {output_dir}.")

In [ ]:
# Set the arguments
#large_las_file = "/content/drive/MyDrive/Artis_AHN3.las"  # Change this to your file path
large_laz_file = "/content/drive/MyDrive/Colab_Notebooks/AWD/datasets/24HN1_25/1_original/24HN1_25.laz"
#output_folder = "/content/drive/MyDrive/Retiled"  # Change this to your desired output folder
output_folder = "/content/drive/MyDrive/Colab_Notebooks/AWD/datasets/24HN1_25/1_original/retiled"
tile_size_meters = 100  # Tile size: 100m × 100m

# Run the tiling function
tile_lidar_las(large_laz_file, output_folder, tile_size_meters)